In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
import glob

digits = [0,1,2,3,4,5,6,7,8,9]
path = 'C:\\Users\\A\\datasets\\MNIST - JPG - training\\'

train = []

for digit in digits:
    digit_path = path+str(digit)+'\*'
    for file in glob.glob(digit_path):
        # make label from filename
        filename = os.path.basename(file)
        #label = filename.split('_')[0]
        train.append([file, str(digit)])

df = pd.DataFrame(train, columns=['file_name', 'text'])

In [ ]:
from sklearn.model_selection import train_test_split
df['text'].value_counts()
df2 = pd.DataFrame()
for digit in digits:
    df_sample = df[df['text'] == str(digit)].sample(n=5000,random_state = 1)
    df2 = df2.append(df_sample)

In [ ]:
# custom label dataset
digits = [0,1,2,3,4,5,6,7,8,9]
path = 'C:\\Users\\A\\datasets\\school_label_dataset\\extract_digits\\'

train = []

for digit in digits:
    digit_path = path+str(digit)+'\*'
    for file in glob.glob(digit_path):
    # make label from filename
        filename = os.path.basename(file)
        #label = filename.split('_')[0]
        train.append([file, str(digit)])

df3 = pd.DataFrame(train, columns=['file_name', 'text'])

In [ ]:
# split / train testset for training

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.15)
# we reset the indices to start from zero
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image

class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text 
        file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(self.root_dir + file_name).convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, 
                                          padding="max_length", 
                                          max_length=self.max_target_length).input_ids
        
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}

        return encoding

In [ ]:
from transformers import TrOCRProcessor

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
train_dataset = IAMDataset(root_dir='',
                           df=train_df,
                           processor=processor)
eval_dataset = IAMDataset(root_dir='',
                           df=test_df,
                           processor=processor)

In [ ]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))

In [ ]:
encoding = train_dataset[0]
for k,v in encoding.items():
    print(k, v.shape)

In [ ]:
# print sample image
image = Image.open(train_dataset.root_dir + train_df['file_name'][0]).convert("RGB")
print('Label: '+train_df['text'][0])
image

In [ ]:
from datasets import load_metric

cer_metric = load_metric("cer")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [ ]:
from transformers import VisionEncoderDecoderModel

model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")

In [ ]:
# parameters

# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 10
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [ ]:
from transformers import default_data_collator
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    num_train_epochs=3,
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    fp16=True, 
    output_dir=".",
    logging_steps=2,
    save_steps=20000,
    eval_steps=5000,
    save_total_limit=1,
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)

trainer.train()

#Step	Training Loss	Validation Loss	Cer
#5000	0.449600	6.391628	2.151000
#10000	0.113700	9.725458	2.002444
#15000	0.291600	8.634416	1.493889
#20000	0.232700	9.673293	1.266778
#25000	0.333000	10.493820	1.084333
#30000	0.042500	10.242682	1.793444
#35000	0.004500	11.528788	2.001000

In [ ]:
model.save_pretrained("model/")